In [123]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

In [124]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [152]:
documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

In [153]:
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())
    
all_words = nltk.FreqDist(all_words)
# print(all_words.most_common(15))
# print(all_words["movie"])

In [194]:
word_features = list(all_words.keys())

In [195]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [199]:
# print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [200]:
training_set = featuresets[:1900]
testing_set = featuresets[1900:]

# print(int(len(featuresets)))
# training_set = featuresets[int(len(featuresets) * .95):int(len(featuresets) * .05)]
# testing_set = featuresets[int(len(featuresets) * .05):int(len(featuresets) * .95)]

# print(featuresets[:50])

# training_set = featuresets[int(len(featuresets) * .05):int(len(featuresets) * .95)]
# testing_set = featuresets[int(len(featuresets) * .95):int(len(featuresets) * .05)]

In [201]:
# classifier = nltk.NaiveBayesClassifier.train(training_set)

## Load an existing classifier.pickle file to the classifier, instead of re-training it every time.
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

## Save the classifier as a .pickle file.
# save_classifier = open("naivebayes.pickle", "wb")
# pickle.dump(classifier, save_classifier)
# save_classifier.close()

In [202]:
print("Original Naive Bayes Classifier accuracy percent: ", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Original Naive Bayes Classifier accuracy percent:  83.0
Most Informative Features
                  avoids = True              pos : neg    =     13.0 : 1.0
               affecting = True              pos : neg    =     11.7 : 1.0
             outstanding = True              pos : neg    =     11.2 : 1.0
                    slip = True              pos : neg    =     11.0 : 1.0
             fascination = True              pos : neg    =     11.0 : 1.0
               strongest = True              pos : neg    =     11.0 : 1.0
              astounding = True              pos : neg    =     10.4 : 1.0
                thematic = True              pos : neg    =     10.4 : 1.0
            manipulation = True              pos : neg    =     10.4 : 1.0
              unbearable = True              neg : pos    =     10.3 : 1.0
               insulting = True              neg : pos    =     10.2 : 1.0
                   sucks = True              neg : pos    =      9.8 : 1.0
                  

In [203]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent: ", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

MNB_classifier accuracy percent:  84.0


In [204]:
BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:",(nltk.classify.accuracy(BNB_classifier, testing_set))*100)

BernoulliNB_classifier accuracy percent: 81.0


In [205]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

LogisticRegression_classifier accuracy percent: 86.0


In [206]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

D:\Applications\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 84.0


In [207]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

SVC_classifier accuracy percent: 49.0


In [208]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

LinearSVC_classifier accuracy percent: 86.0


In [209]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier accuracy percent: 87.0
